In [7]:
from HLsearch import *
from scipy.integrate import solve_ivp
import numpy as np
from sympy import symbols, var, diff, simplify, collect,solve
from sympy.utilities.lambdify import lambdify, implemented_function

from operator import add,sub,mul

import itertools
import scipy.io as sio
data = sio.loadmat('./Data/springmass.mat')

X = np.real(data['y'])
Xdot = np.real(data['dy'])

data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('x{}, x{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)
expr = buildFunctionExpressions(2,X.shape[1],data_description,use_sine=True)
print(expr)
Theta = buildTimeSerieMatrixFromFunctions(X,expr, data_description)

Gamma = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr,data_description)


u, s, vh = np.linalg.svd(0*Theta-Gamma,full_matrices=False)

xi = vh.T[:,-1]

# print('{} coefficients ='.format(xi.shape[0]),xi)
print('Now drop off small coefficients')
Hamiltonian,terms = generateSimplifiedExpression(xi,expr)

print('H = ',Hamiltonian)

Variables are: (x0, x0_t)
['x0', 'x0_t', 'sin(x0)', 'cos(x0)', 'x0**2', 'x0*x0_t', 'x0_t**2', 'x0*sin(x0)', 'x0_t*sin(x0)', 'sin(x0)**2', 'x0*cos(x0)', 'x0_t*cos(x0)', 'sin(x0)*cos(x0)', 'cos(x0)**2']
Now drop off small coefficients
H =  -0.18499615071074077*x0**2 - 0.007399846028685876*x0_t**2


In [8]:
print('Now start calculating 2 spring-mass')

data = sio.loadmat('./Data/springmass2_k_is_25_and_9.mat')

X = np.real(data['y'])
Xdot = np.real(data['dy'])
data_description = ()
for i in range(round(X.shape[1]/2)):
    data_description = data_description + symbols('x{}, x{}_t'.format(i,i))
print('Variables are:',data_description)
data_description_sym = data_description
data_description = list(str(descr) for descr in data_description)

expr_new = buildFunctionExpressions(2,X.shape[1],data_description,use_sine=False)
Theta_new = buildTimeSerieMatrixFromFunctions(X,expr_new, data_description)
Gamma_old = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr    ,data_description)
Gamma_new = buildTimeDerivativeMatrixFromFunctions(X,Xdot,expr_new,data_description)

stored_indices = tuple(expr_new.index(str(f)) for f in terms)

print('Keeping terms: ',[expr_new[i] for i in stored_indices])
elements = tuple(x for x in range(len(expr_new)) if x not in stored_indices)
print(stored_indices)
Hamiltonian_old = Hamiltonian

combi_number = 11
indices = itertools.combinations(elements, combi_number)
zero_in_xi = np.array([collect(Hamiltonian_old,term).coeff(term) for term in terms],dtype=np.float32)
for _ in itertools.repeat(None, combi_number):
    zero_in_xi = np.insert(zero_in_xi,0,0)
# print(zero_in_xi)

goodHamiltonian={}

for count,index in enumerate(indices):
# index_tup = (2,5,16,25)
    index_tup = index + stored_indices
    xi_new = STRidge(Gamma_new[:,index_tup], -1*np.dot(Gamma_old,xi),0.0,1000, 10**-12, print_results=False)
    Hamiltonian = Hamiltonian_old+generateExpression(xi_new,[expr_new[i] for i in index_tup],threshold=1e-6)
    if np.linalg.norm(xi_new+zero_in_xi,2) < 1e-5: continue
    energyFunc = lambdify(data_description,Hamiltonian,'numpy')
    energy = np.array([energyFunc(*X[i,:]) for i in range(X.shape[0])])
    # var = np.var(energy)
    if np.var(energy)<1e-10 and abs(np.mean(energy))>1e-3:
        temp_terms = [sympify(expr_new[i]) for j,i in enumerate(index_tup) if abs(xi_new[j])>1e-8]
        Lagrangian = findLagrangianFromHamiltonian(Hamiltonian,temp_terms,data_description_sym,threshold=1e-6)
        if Lagrangian is not None and Lagrangian != '':
            goodHamiltonian[str(simplify(Hamiltonian))] = Lagrangian
            print('Found good result at ',count,'th trial: ',index_tup)


Now start calculating 2 spring-mass
Variables are: (x0, x0_t, x1, x1_t)
Keeping terms:  ['x0**2', 'x0_t**2']
(4, 6)
Found good result at  0 th trial:  (0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 12, 4, 6)
Found good result at  1 th trial:  (0, 1, 2, 3, 5, 7, 8, 9, 10, 11, 13, 4, 6)
Found good result at  2 th trial:  (0, 1, 2, 3, 5, 7, 8, 9, 10, 12, 13, 4, 6)
Found good result at  3 th trial:  (0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 4, 6)
Found good result at  4 th trial:  (0, 1, 2, 3, 5, 7, 8, 10, 11, 12, 13, 4, 6)
Found good result at  5 th trial:  (0, 1, 2, 3, 5, 7, 9, 10, 11, 12, 13, 4, 6)
Found good result at  6 th trial:  (0, 1, 2, 3, 5, 8, 9, 10, 11, 12, 13, 4, 6)
Found good result at  7 th trial:  (0, 1, 2, 3, 7, 8, 9, 10, 11, 12, 13, 4, 6)
Found good result at  8 th trial:  (0, 1, 2, 5, 7, 8, 9, 10, 11, 12, 13, 4, 6)
Found good result at  9 th trial:  (0, 1, 3, 5, 7, 8, 9, 10, 11, 12, 13, 4, 6)
Found good result at  10 th trial:  (0, 2, 3, 5, 7, 8, 9, 10, 11, 12, 13, 4, 6)
Found good result 

In [9]:
def generate_data(func, time, init_values):
    sol = solve_ivp(func,[time[0],time[-1]],init_values,t_eval=time, method='RK45',rtol=1e-10,atol=1e-10)
    return sol.y.T, np.array([func(0,sol.y.T[i,:]) for i in range(sol.y.T.shape[0])],dtype=np.float64)

def twoSpringMassForced(t,y,f=1.0,k1=25,k2=9):
    x1,x1_t,x2,x2_t = y
    x1_2t = -(k1+k2)*x1 +k2*x2
    x2_2t = -k2*(x2-x1) + f
    return x1_t,x1_2t,x2_t,x2_2t

t = np.arange(0,0.1,0.01)
y0=np.array([1.0, 0,0, 0])
Xf,Xfdot = generate_data(twoSpringMassForced,t,y0)
best_var = np.inf
best_L = None
best_H = None

for H,L in goodHamiltonian.items():
    # print('Found Hamiltonian is ',H)
    # print('Lagrangian is ',L)
    dLdq_expr = diff(L, 'x1')
    dLdqdot_expr = diff(L, 'x1_t')
    dLdq = buildTimeSerieFromFunction(Xf,dLdq_expr,data_description_sym)
    d_dLdqdot_dt = buildTimeDerivativeSerieFromFunction(Xf,Xfdot,dLdqdot_expr,data_description_sym)
    fCal = d_dLdqdot_dt-dLdq
    # print(np.var(fCal))
    k = 1.0/np.mean(fCal)
    # print('Hamiltonian is ',k*H)
    # print('True Lagrangian is ',k*L)
    # print('')
    if np.var(fCal)<best_var:
        best_var = np.var(fCal)
        best_L = k*L
        best_H = k*sympify(H)

print('H = ',best_H)
print('L = ',best_L)

H =  16.999999999205109*x0**2 - 9.000000000000068*x0*x1 + 0.5000000000000042*x0_t**2 + 4.500000000000033*x1**2 + 0.5000000000000036*x1_t**2
L =  -16.9999999992051*x0**2 + 9.00000000000006*x0*x1 + 0.500000000000004*x0_t**2 - 4.50000000000003*x1**2 + 0.500000000000003*x1_t**2
